• KLM 56377143
• AirFrance 106062176
• British_Airways 18332190
• AmericanAir 22536055
• Lufthansa 124476322
• AirBerlin 26223583
• AirBerlin assist 2182373406
• easyJet 38676903
• RyanAir 1542862735
• SingaporeAir 253340062
• Qantas 218730857
• EtihadAirways 45621423
• VirginAtlantic 20626359

In [2]:
 import mysql.connector 

In [3]:
db = mysql.connector.connect(host="127.0.0.1", user="root", password="",database="dbl")
cursor=db.cursor()

In [4]:
cursor.execute("SELECT * FROM `dayone` ORDER BY timestamp DESC") 
dayone=cursor.fetchall() #store all tweets of one day from new to old in dayone

In [8]:
cursor.execute("DROP TABLE `conversations`") 


In [4]:
cursor.execute("SELECT * FROM `tweets` ORDER BY timestamp DESC")
total=cursor.fetchall() #store all tweets from new to old in total

In [5]:
cursor.execute("SELECT COUNT(*) FROM `tweets` t")
tot_numb=cursor.fetchall()[0][0]
print(tot_numb) #return the total number of tweets

6094135


# Ideas for conversations implementation
loop over SQL table and store what messages are in what conversations in either SQL or dictionaries
Use the attributes id, conversation start timestamp (which keeps getting updated when adding new values), conversation end timestamp(which will stay constant after creating it, since we start with its final tweet), has an airline participated (when adding a tweet to a conversation check if it is from an airline or not if the conversation does not already have a participating airline), length of conversation (increment by 1 every time a tweet is added to a conversation).

for all tweet_ids <br>
&emsp; if id in dictionary <br>
&emsp; &emsp; link this to those conversations in dictionary<br>
&emsp; &emsp; update the info for those conversations<br>
&emsp; else<br>
&emsp; &emsp; create a new conversation with values from this tweet<br>
&emsp; &emsp; create a tuple linking this tweet to this conversation<br>
&emsp; if in_reply_to_id not zero<br>
&emsp; &emsp; add conversation id from this tweet to that id's dictionary list<br>

CREATE TABLE table_name (<br>
    column1 datatype,<br>
    column2 datatype,<br>
    column3 datatype,<br>
   ....<br>
); 

In [25]:
def create_tables():
    """This function will create a new table in the MySQL environment called conversations with the following attributes:
    conversation ID, conversation start time, conversation end time, has an airline participated, length of conversations,
    and the response time. Furthermore, also create a table that links tweets to conversations they are a part of."""
    
    cursor.execute("""CREATE TABLE `dbl`.`conversations`(id MEDIUMINT NOT NULL, Tstart BIGINT NOT NULL, Tend BIGINT NOT NULL,
    Airline SMALLINT NOT NULL,length SMALLINT NOT NULL, Tresponse MEDIUMINT, PRIMARY KEY (id))""")
    
    cursor.execute("""CREATE TABLE `dbl`.`part_of`(tID BIGINT NOT NULL, cID MEDIUMINT NOT NULL, PRIMARY KEY(tID, cID), 
    FOREIGN KEY (tID) REFERENCES `tweets`(id), FOREIGN KEY (cID) REFERENCES `conversations`(id))""")
    return 'done'

def drop_tables():
    """Drop the table called conversations and the one called part_of"""
    cursor.execute("DROP TABLE `part_of`")
    cursor.execute("DROP TABLE `conversations`")

    return 'dropped'

'done'

INSERT INTO table(c1,c2,...) <br>
VALUES <br>
   (v11,v12,...), <br>
   (v21,v22,...), <br>
    ... <br>
   (vnn,vn2,...); <br>

In [6]:
def fill_tables (tweets: list) -> None:
    """This function will insert into the tables 'part of' and 'conversations' the correct information given an input
        SQL table tweets.
        """
    tracker = 0
    #store what conversations a certain tweet_id will have to become a member of, because a reply to it is in it, in this dict.

    member_of = dict()

    #increment this whenever you create a new conversation, to ensure there are no duplicate ids.
    conv_id = 0
    
    for i in tweets:
        
        if i[0] in member_of:
            convs = member_of[i[0]]
            member_of.pop(i[0])
            #link i[0] to all cIds in the convs list in part of table 
            #update information about conversation in member_of2
            tracker += 1
        else:
            convs = [conv_id] #store what conversations tweet i has become a part of
            #create a new conversation in the conversations table with the information from tweet i
            #link i[0] to the newly created conversation with conv_id in part of table.
            conv_id += 1 #increment the conversation id to ensure no duplicate conversation ids occur
            
        if i[2] != 0: #if this tweet replies to someone
            if i[2] in member_of:   #if the tweet this replies to already has another reply, add the new conversations to dict
                                    #and keep old one.
                member_of[i[2]] = member_of[i[2]] + convs
                
            else:
                member_of[i[2]] = convs
                
    return conv_id, tracker#,member_of)

In [7]:
FT_tot = fill_tables(total)
print(FT_tot)

(5137867, 956268)


In [8]:
if sum(FT_tot) == tot_numb:
    print("Everything worked correctly")

Everything worked correctly


In [9]:
FT_ex = fill_tables([[0,1,1],[1,2,2],[2,3,3],[3,4,4],[4,5,5],[7,1,4235],[5,1,0],[9,1,0]])
print(FT_ex)

(3, 5)


In [10]:
FT_dayone = fill_tables(dayone)
print(FT_dayone)

(10690, 2307)


In [11]:
#%%timeit
#fill_tables(total)

Normal: <br>
1k: 215 microseconds <br>
10k: 2.86 ms <br>
100k: 32.3 ms <br>
6 million (all): 3.19 s

In [12]:
repliers = 0
for i in total:
    if i[2] != 0:
        repliers += 1

In [13]:
print(f"We have {repliers} replying tweets and we have {956268} tweets that are replied to")

We have 1788039 replying tweets and we have 956268 tweets that are replied to
